# Visualization Factory

## Overview
- **What**: How to use `StyleManager`, `Theme`, and `FigureFactory` to create consistent, professional plots across all reports.
- **Prerequisites**: [getting-started/01_setup_verification](../getting-started/01_setup_verification.ipynb)
- **Estimated runtime**: < 1 minute
- **Audience**: [Developer] / [Practitioner]

## Key Concepts
- **StyleManager**: Centralized theme and style configuration
- **FigureFactory**: Standardized plot creation with automatic styling
- **Themes**: DEFAULT, COLORBLIND, PRESENTATION, MINIMAL, PRINT

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/visualization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ergodic_insurance.visualization import StyleManager, Theme, FigureFactory

np.random.seed(42)

print("Visualization Factory loaded successfully!")

## 1. StyleManager: Theme Management

The `StyleManager` provides centralized control over colors, fonts, and figure sizing for all plots.

In [ ]:
# Create style manager and explore available themes
style_mgr = StyleManager(theme=Theme.DEFAULT)

print("Available Themes:")
print("=" * 40)
for theme in Theme:
    print(f"  - {theme.value}: {theme.name}")

colors = style_mgr.get_colors()
fonts = style_mgr.get_fonts()
figure = style_mgr.get_figure_config()

print(f"\nCurrent Theme Configuration (DEFAULT):")
print("=" * 40)
print(f"  Primary Color:   {colors.primary}")
print(f"  Font Family:     {fonts.family}")
print(f"  Base Font Size:  {fonts.size_base}")
print(f"  Blog Figure Size: {figure.size_blog}")
print(f"  Web DPI:         {figure.dpi_web}")
print(f"  Print DPI:       {figure.dpi_print}")

## 2. FigureFactory: Standardized Plot Creation

`FigureFactory` creates various plot types with automatic styling applied from the selected theme.

In [ ]:
# Create a line plot using the factory
factory = FigureFactory(theme=Theme.DEFAULT)

years = np.arange(2015, 2025)
revenue = np.array([10, 12, 13, 15, 14, 16, 18, 20, 22, 24])
profit = revenue * np.random.uniform(0.08, 0.12, len(revenue))

fig, ax = factory.create_line_plot(
    x_data=years,
    y_data={"Revenue": revenue, "Profit": profit},
    title="Company Performance Over Time",
    x_label="Year",
    y_label="Amount ($M)",
    size_type="blog",
    show_legend=True,
    markers=True,
)
factory.format_axis_currency(ax, axis="y", abbreviate=True)
plt.show()

## 3. Theme Comparison

The same data rendered with four different themes shows how easy it is to switch visual styles for different audiences.

In [ ]:
# Compare themes on the same sample distribution
data = np.random.randn(1000) * 2 + 5

themes_to_compare = [Theme.DEFAULT, Theme.COLORBLIND, Theme.PRESENTATION, Theme.MINIMAL]
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, theme in enumerate(themes_to_compare):
    ax = axes.flat[idx]
    theme_factory = FigureFactory(theme=theme, auto_apply=False)
    theme_factory.style_manager.apply_style()
    colors = theme_factory.style_manager.get_colors()

    ax.hist(data, bins=30, color=colors.primary, alpha=0.7,
            edgecolor="black", linewidth=0.5)

    mean_val = np.mean(data)
    median_val = np.median(data)
    ax.axvline(mean_val, color=colors.warning, linestyle="--",
               linewidth=2, label=f"Mean: {mean_val:.2f}")
    ax.axvline(median_val, color=colors.success, linestyle="--",
               linewidth=2, label=f"Median: {median_val:.2f}")

    ax.set_title(f"Theme: {theme.value.upper()}")
    ax.set_xlabel("Value")
    ax.set_ylabel("Frequency")
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle("Same Data with Different Themes", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 4. Insurance Claims Analysis Dashboard

Combining multiple subplot types into a single dashboard with the `PRESENTATION` theme for executive-quality output.

In [ ]:
# Generate sample insurance claims data
np.random.seed(42)
claim_sizes = np.random.lognormal(mean=10, sigma=2, size=1000) * 100_000

factory = FigureFactory(theme=Theme.PRESENTATION)

fig, axes = factory.create_subplots(
    rows=2, cols=2,
    size_type="technical",
    title="Insurance Claims Analysis Dashboard",
    subplot_titles=[
        "Claim Size Distribution",
        "Exceedance Probability",
        "Claims by Size Category",
        "Risk Metrics",
    ],
)

# Panel 1: Histogram of claim sizes
ax1 = axes[0, 0]
ax1.hist(claim_sizes / 1e6, bins=50,
         color=factory.style_manager.get_colors().primary,
         alpha=0.7, edgecolor="black", linewidth=0.5)
ax1.set_xlabel("Claim Size ($M)")
ax1.set_ylabel("Frequency")
ax1.grid(True, alpha=0.3)

# Panel 2: Exceedance curve
ax2 = axes[0, 1]
sorted_claims = np.sort(claim_sizes)[::-1]
exceedance_prob = np.arange(1, len(sorted_claims) + 1) / len(sorted_claims)
ax2.semilogy(sorted_claims / 1e6, exceedance_prob,
             color=factory.style_manager.get_colors().warning, linewidth=2)
ax2.set_xlabel("Claim Size ($M)")
ax2.set_ylabel("Exceedance Probability")
ax2.grid(True, alpha=0.3, which="both")

# Panel 3: Bar chart of claim categories
ax3 = axes[1, 0]
categories = ["Small\n(<$1M)", "Medium\n($1-10M)", "Large\n($10-50M)", "Cat.\n(>$50M)"]
counts = [
    np.sum(claim_sizes < 1e6),
    np.sum((claim_sizes >= 1e6) & (claim_sizes < 10e6)),
    np.sum((claim_sizes >= 10e6) & (claim_sizes < 50e6)),
    np.sum(claim_sizes >= 50e6),
]
bars = ax3.bar(categories, counts,
               color=factory.style_manager.get_colors().series[:4], alpha=0.8)
ax3.set_ylabel("Number of Claims")
ax3.grid(True, alpha=0.3, axis="y")
for bar, count in zip(bars, counts):
    ax3.text(bar.get_x() + bar.get_width() / 2., bar.get_height(),
             str(int(count)), ha="center", va="bottom")

# Panel 4: Risk metrics
ax4 = axes[1, 1]
metrics = {
    "Mean": np.mean(claim_sizes) / 1e6,
    "Median": np.median(claim_sizes) / 1e6,
    "VaR(95%)": np.percentile(claim_sizes, 95) / 1e6,
    "VaR(99%)": np.percentile(claim_sizes, 99) / 1e6,
    "Max": np.max(claim_sizes) / 1e6,
}
y_pos = np.arange(len(metrics))
bars = ax4.barh(y_pos, list(metrics.values()),
                color=factory.style_manager.get_colors().secondary, alpha=0.8)
ax4.set_yticks(y_pos)
ax4.set_yticklabels(list(metrics.keys()))
ax4.set_xlabel("Value ($M)")
ax4.grid(True, alpha=0.3, axis="x")
for bar, val in zip(bars, metrics.values()):
    ax4.text(val, bar.get_y() + bar.get_height() / 2.,
             f"${val:.1f}M", ha="left", va="center", fontsize=10)

plt.tight_layout()
plt.show()

print("\nRisk Summary Statistics:")
print("=" * 40)
for metric, value in metrics.items():
    print(f"  {metric:12s}: ${value:,.1f}M")

## 5. Colorblind-Friendly Visualization

The `COLORBLIND` theme provides accessible color palettes. Especially important for publications and public-facing reports.

In [ ]:
# Colorblind-friendly multi-series line plot
factory = FigureFactory(theme=Theme.COLORBLIND)

months = pd.date_range("2023-01", periods=12, freq="M")
products = {
    "General Liability": np.cumsum(np.random.randn(12) * 0.5 + 1.0) * 100_000,
    "Property": np.cumsum(np.random.randn(12) * 0.3 + 0.8) * 100_000,
    "Workers Comp": np.cumsum(np.random.randn(12) * 0.4 + 0.6) * 100_000,
    "Auto": np.cumsum(np.random.randn(12) * 0.2 + 0.5) * 100_000,
    "Cyber": np.cumsum(np.random.randn(12) * 0.6 + 0.3) * 100_000,
}

fig, ax = factory.create_line_plot(
    x_data=months,
    y_data=products,
    title="Cumulative Premiums by Product Line (Colorblind-Friendly)",
    x_label="Month",
    y_label="Cumulative Premium",
    size_type="blog",
    show_legend=True,
    markers=True,
)
factory.format_axis_currency(ax, axis="y", abbreviate=True)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
plt.tight_layout()
plt.show()

colors = factory.style_manager.get_colors()
print("\nColorblind-Friendly Palette:")
for i, color in enumerate(colors.series[:5]):
    print(f"  Series {i + 1}: {color}")

## 6. Print-Ready Report Generation

The `PRINT` theme creates high-DPI figures optimized for inclusion in printed reports and LaTeX documents.

In [ ]:
# Print-quality bar chart
factory = FigureFactory(theme=Theme.PRINT)

quarters = ["Q1", "Q2", "Q3", "Q4"]
metrics_data = {
    "Premium": [25, 28, 32, 35],
    "Claims": [15, 18, 20, 19],
    "Profit": [10, 10, 12, 16],
}

fig, ax = factory.create_bar_plot(
    categories=quarters,
    values=metrics_data,
    title="Quarterly Performance Summary (Print Quality)",
    x_label="Quarter",
    y_label="Amount ($M)",
    size_type="technical",
    dpi_type="print",
    show_values=True,
)

textstr = "\n".join([
    "Key Insights:",
    "  Premium growth: +40% YoY",
    "  Loss ratio improved to 54%",
    "  Profit margin: 35%",
])
props = dict(boxstyle="round", facecolor="wheat", alpha=0.5)
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment="top", bbox=props)

plt.tight_layout()
plt.show()
print("Figure rendered with print quality (300 DPI)")

## 7. Custom Configuration

Customize colors and fonts for corporate branding, then save/load configurations for reuse.

In [ ]:
# Override colors and fonts for a custom corporate theme
custom_style = StyleManager(theme=Theme.DEFAULT)
custom_style.update_colors({
    "primary": "#003366",
    "secondary": "#FF6600",
    "accent": "#00AA44",
})
custom_style.update_fonts({
    "family": "Helvetica",
    "size_base": 12,
    "size_title": 16,
})

custom_factory = FigureFactory(style_manager=custom_style)

categories = ["Product A", "Product B", "Product C", "Product D"]
values = [45, 38, 52, 41]
colors_custom = custom_style.get_colors()

fig, ax = custom_factory.create_figure(size_type="blog", title="Custom Corporate Styling Demo")
bars = ax.bar(
    categories, values,
    color=[colors_custom.primary, colors_custom.secondary,
           colors_custom.accent, colors_custom.warning],
    alpha=0.8,
)
ax.set_ylabel("Performance Score")
ax.set_title("Product Performance Comparison")
ax.grid(True, alpha=0.3, axis="y")
for bar in bars:
    ax.text(bar.get_x() + bar.get_width() / 2., bar.get_height(),
            str(int(bar.get_height())), ha="center", va="bottom")
plt.tight_layout()
plt.show()

# Save and reload the configuration
config_path = "../_utilities/custom_style_config.yaml"
custom_style.save_config(config_path)

loaded_style = StyleManager()
loaded_style.load_config(config_path)
loaded_colors = loaded_style.get_colors()

print("\nLoaded Custom Configuration:")
print(f"  Primary:   {loaded_colors.primary}")
print(f"  Secondary: {loaded_colors.secondary}")
print(f"  Accent:    {loaded_colors.accent}")

## 8. Integration with the Visualization Module

The factory integrates seamlessly with the existing `ergodic_insurance.visualization` module and its built-in plot functions.

In [ ]:
# Use existing visualization functions with factory styling
from ergodic_insurance import visualization as viz

np.random.seed(42)
losses = np.random.lognormal(mean=10, sigma=1.5, size=5000) * 100_000

fig = viz.plot_loss_distribution(
    losses=losses,
    title="Loss Distribution with Factory Styling",
    bins=50,
    show_metrics=True,
    var_levels=[0.95, 0.99],
    use_factory=True,
    theme=Theme.PRESENTATION,
)
plt.show()
print("Factory integration with existing visualization module successful!")

## Key Takeaways

- **StyleManager** centralizes theme management: colors, fonts, sizes, DPI settings.
- **FigureFactory** produces standardized plots with automatic styling from any theme.
- Five built-in themes cover common audiences: DEFAULT, COLORBLIND, PRESENTATION, MINIMAL, PRINT.
- Custom themes can be saved to YAML and shared across notebooks.
- The factory is backward-compatible with existing `ergodic_insurance.visualization` functions.

## Next Steps

- [02_executive_dashboards](02_executive_dashboards.ipynb) -- executive-level dashboard visualizations
- [03_ergodic_visualizations](03_ergodic_visualizations.ipynb) -- ergodic-specific visualization patterns
- [05_ruin_analysis_plots](05_ruin_analysis_plots.ipynb) -- ruin cliff and ROE-ruin frontier plots